In [ ]:
!pip install openai-whisper
!pip install coqui-tts
!pip install tmdbv3api



In [62]:
from google.colab import userdata

#Componente ASR:
import whisper

model = whisper.load_model("base")
result = model.transcribe("/var/Entradas_Audio/not_Padri.wav")
transcripcion = result['text']
print("Transcripción entrante:", transcripcion)

#--------------------------------------------------------------------------------------------------------------------------------
#Gestor de dialogo:

#Integracion de la API
from tmdbv3api import TMDb, Movie

tmdb = TMDb()
tmdb.language = "es-ES"  # En español
tmdb.api_key = userdata.get('API_KEY')
movie = Movie()

#Elimino caracteres no alfabéticos (excepto los espacios)
import re

def limpiar_texto(texto):
    texto_limpio = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚñÑ ]', '', texto)
    return texto_limpio

#Paso de numero a texto
def numero_a_texto(num):
    #Defino los números básicos
    unidades = ["", "uno", "dos", "tres", "cuatro", "cinco", "seis", "siete", "ocho", "nueve"]
    decenas = ["", "diez", "veinte", "treinta", "cuarenta", "cincuenta", "sesenta", "setenta", "ochenta", "noventa"]
    centenas = ["", "cien", "doscientos", "trescientos", "cuatrocientos", "quinientos", "seiscientos", "setecientos", "ochocientos", "novecientos"]

    #Convierto el número a entero y decimal
    entero, decimal = str(round(num, 2)).split('.') if '.' in str(num) else (str(num), "00")

    def convertir_entero(num):
        #Convierto la parte entera a texto
        num = int(num)
        if num == 0:
            return "cero"
        elif num == 100:
            return "cien"
        elif num < 10:
            return unidades[num]
        elif num < 20:
            if num == 11:
                return "once"
            return f"dieci{unidades[num-10]}"
        elif num < 30:
            if num == 20:
                return "veinte"
            return f"veinti{unidades[num-20]}"
        elif num < 100:
            decena = num // 10
            unidad = num % 10
            return f"{decenas[decena]} y {unidades[unidad]}" if unidad > 0 else decenas[decena]
        elif num < 1000:
            centena = num // 100
            resto = num % 100
            if resto == 0:
                return centenas[centena]
            return f"{centenas[centena]} {convertir_entero(str(resto))}"
        else:
            return "número fuera de rango"

    def convertir_decimal(decimal):
        #Convierto la parte decimal a texto
        if decimal == "00":
            return ""
        else:
            return f"coma {convertir_entero(decimal[0])}"


    #Convierto la parte entera y decimal
    texto_entero = convertir_entero(entero)
    texto_decimal = convertir_decimal(decimal)

    return f"{texto_entero} {texto_decimal}".strip()

def procesar_consulta(intencion, nombre_pelicula):
    #Busco la película en TMDB
    resultados = movie.search(nombre_pelicula)
    if not resultados:
        return f"No he encontrado información sobre '{nombre_pelicula}'."

    pelicula = resultados[0]  #Asumo el primer resultado como el correcto

    #Genero la respuesta según la intención
    if intencion == "sinopsis":
        return f"La sinopsis de '{pelicula.title}' es: {pelicula.overview}"
    elif intencion == "reparto":
        creditos = movie.credits(pelicula.id)  #Llamo al endpoint de créditos
        #Valido si 'cast' existe y contiene datos
        if creditos.get('cast'):
            #Obtengo nombres y personajes de los actores
            reparto = []
            for actor in creditos['cast']:  #Itero sobre cada actor (que es un diccionario)
                nombre_actor = actor.get('name', 'Desconocido')  #Obtengo el nombre del actor
                personaje = actor.get('character', None)  #Obtengo el personaje del actor (si existe)
                if personaje:
                    reparto.append(f"{nombre_actor} como {personaje}")
                else:
                    reparto.append(nombre_actor)
            return f"El reparto principal de '{pelicula.title}' incluye: {', '.join(reparto)}."
        else:
            return f"No he encontrado información sobre el reparto de '{pelicula.title}'."
    elif intencion == "director":
        creditos = movie.credits(pelicula.id)
        directores = [persona['name'] for persona in creditos.crew if persona['job'] == "Director"]
        return f"El director de '{pelicula.title}' es: {', '.join(directores)}."
    elif intencion == "puntuación":
        return f"'{pelicula.title}' tiene una puntuación de {numero_a_texto(pelicula.vote_average)} en TMDB."
    else:
        return "Lo siento, no he entendido ese numerín."

def interpretar_intencion(texto):
    #Palabras clave para identificar la intención
    palabras_clave = {
        "sinopsis": ["sinopsis", "trata", "de qué trata", "historia", "de qué va", "resumen", "de que va"],
        "reparto": ["actúa", "actúan", "actores", "reparto", "elenco", "protagonistas", "actrices", "protas"],
        "director": ["dirigió", "director", "dirige", "dirigen", "directores"],
        "puntuación": ["calificación", "rating", "puntuación", "nota"]
    }

    #Determino la intención
    intencion = None
    for clave, palabras in palabras_clave.items():
        if any(palabra in texto.lower() for palabra in palabras):
            intencion = clave
            break

    #Extraigo el nombre de la película (asumiendo que es lo último mencionado)
    palabras = texto.split()
    nombre_pelicula = " ".join(palabras[-2:])  #Las últimas palabras como nombre

    return intencion, nombre_pelicula


intencion, nombre_pelicula = interpretar_intencion(transcripcion)
respuesta = limpiar_texto(procesar_consulta(intencion, nombre_pelicula))
print(respuesta)

#--------------------------------------------------------------------------------------------------------------------------------
#Implementación del componente TTS (usando Coqui TTS)
from TTS.api import TTS
tts = TTS(model_name="tts_models/es/css10/vits", progress_bar=True, gpu=False)
tts.tts_to_file(text=respuesta, file_path="respuesta.wav")

#Guardado del archivo generado:
from google.colab import files
files.download("respuesta.wav")


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
/usr/local/li

Transcripción entrante:  ¿Qué nota tiene el padrino?
El padrino tiene una puntuación de ocho coma seis en TMDB


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>